In [7]:
import cv2
import os

# Ruta de la carpeta con imágenes
image_folder = r"C:\Users\Edu\Documents\GitHub\DeepPrune\images"

# Obtener la lista de archivos en la carpeta
image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

# Verificar y cargar imágenes
images = []
for img_path in image_files:
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error al cargar la imagen: {img_path}")
    else:
        images.append(img)

# Comprobar si todas las imágenes se cargaron correctamente
if len(images) == 0:
    print("No se encontraron imágenes válidas en la carpeta.")
else:
    print(f"Se cargaron correctamente {len(images)} imágenes.")


Se cargaron correctamente 72 imágenes.


In [ ]:
import cv2
import numpy as np
import os

# Ruta de la carpeta con imágenes
image_folder = r"C:\Users\Edu\Documents\GitHub\DeepPrune\images"

# Obtener lista ordenada de imágenes
image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

# Cargar imágenes en una lista
images = [cv2.imread(img_path) for img_path in image_files]

# Verificar si todas las imágenes se cargaron correctamente
if any(img is None for img in images):
    raise ValueError("Una o más imágenes no se cargaron correctamente.")

# Inicializar detector SIFT
sift = cv2.SIFT_create()

# Parámetros del matcher FLANN
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

# Lista para almacenar imágenes a conservar
kept_images = []

# Umbral para considerar redundante una imagen intermedia (ajustable)
OVERLAP_THRESHOLD = 80  # 80% de coincidencia entre imagen 1 y 3

# Evaluar cada grupo de tres imágenes consecutivas
for i in range(len(images) - 2):
    img1, img2, img3 = images[i], images[i + 1], images[i + 2]

    # Convertir a escala de grises
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    gray3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

    # Detectar características clave y descriptores
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)
    kp3, des3 = sift.detectAndCompute(gray3, None)

    # Si no hay descriptores en una imagen, saltarla
    if des1 is None or des2 is None or des3 is None:
        continue

    # Comparar imagen 1 con imagen 3
    matches_13 = flann.knnMatch(des1, des3, k=2)

    # Aplicar la regla de Lowe para filtrar coincidencias
    good_matches_13 = [m for m, n in matches_13 if m.distance < 0.7 * n.distance]

    # Calcular porcentaje de coincidencia entre 1 y 3
    match_percentage_13 = len(good_matches_13) / len(matches_13) * 100 if matches_13 else 0

    print(f"Superposición entre {image_files[i]} y {image_files[i+2]}: {match_percentage_13:.2f}%")

    # Decidir si eliminar la imagen intermedia
    if match_percentage_13 > OVERLAP_THRESHOLD:
        print(f"Descartando {image_files[i+1]} (redundante entre {image_files[i]} y {image_files[i+2]})")
    else:
        kept_images.append(image_files[i+1])  # Mantener la imagen intermedia si no es redundante

# Agregar la primera y la última imagen a la lista final
kept_images.insert(0, image_files[0])  # Asegurar que la primera imagen se conserva
kept_images.append(image_files[-1])  # Asegurar que la última imagen se conserva

print(f"\nTotal de imágenes conservadas: {len(kept_images)}")
print("Imágenes seleccionadas:")
for img in kept_images:
    print(img)


Superposición entre C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142901_0001_V.JPG y C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142903_0003_V.JPG: 28.00%
Superposición entre C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142902_0002_V.JPG y C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142903_0004_V.JPG: 16.48%
Superposición entre C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142903_0003_V.JPG y C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142904_0005_V.JPG: 10.56%
Superposición entre C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142903_0004_V.JPG y C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142905_0006_V.JPG: 10.96%
Superposición entre C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142904_0005_V.JPG y C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142905_0007_V.JPG: 11.46%
Superposición entre C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_202310181

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000002089ED30950>>
Traceback (most recent call last):
  File "c:\Users\Edu\Documents\GitHub\DeepPrune\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Superposición entre C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142938_0055_V.JPG y C:\Users\Edu\Documents\GitHub\DeepPrune\images\DJI_20231018142940_0057_V.JPG: 16.05%
